In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from scipy import sparse
from lightfm import LightFM
from sklearn.metrics.pairwise import cosine_similarity
from lightfm.data import Dataset
import ast
import re

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/opt/anaconda3/lib/python3.7/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [2]:
df_review = pd.read_csv('Yelp_Dataset_Reviews_new.csv')
#.drop(['Unnamed: 0'], axis=1)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
print(df_review.dtypes)
print('')
print(type(df_review.text))

Unnamed: 0        int64
Unnamed: 0.1     object
review_id        object
user_id          object
business_id      object
rating          float64
useful          float64
funny           float64
cool            float64
text             object
date             object
year            float64
dtype: object

<class 'pandas.core.series.Series'>


In [4]:
df_review.head()

,Unnamed: 0,Unnamed: 0.1,review_id,user_id,business_id,rating,useful,funny,cool,text,date,year
0,0,0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,6.0,1.0,0.0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36,2013.0
1,1,1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5.0,0.0,0.0,0.0,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33,2017.0
2,2,2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5.0,3.0,0.0,0.0,I have to say that this office really has it t...,2016-11-09 20:09:03,2016.0
3,3,3,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5.0,0.0,0.0,0.0,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38,2018.0
4,4,4,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1.0,7.0,0.0,0.0,Today was my second out of three sessions I ha...,2018-01-30 23:07:38,2018.0


# tokenize text

In [5]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, SnowballStemmer, RegexpStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.summarization import summarize, keywords

In [6]:
df_review.text = df_review.text.str.lower()
df_review.head()

,Unnamed: 0,Unnamed: 0.1,review_id,user_id,business_id,rating,useful,funny,cool,text,date,year
0,0,0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,6.0,1.0,0.0,total bill for this horrible service? over $8g...,2013-05-07 04:34:36,2013.0
1,1,1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5.0,0.0,0.0,0.0,i *adore* travis at the hard rock's new kelly ...,2017-01-14 21:30:33,2017.0
2,2,2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5.0,3.0,0.0,0.0,i have to say that this office really has it t...,2016-11-09 20:09:03,2016.0
3,3,3,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5.0,0.0,0.0,0.0,went in for a lunch. steak sandwich was delici...,2018-01-09 20:56:38,2018.0
4,4,4,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1.0,7.0,0.0,0.0,today was my second out of three sessions i ha...,2018-01-30 23:07:38,2018.0


In [7]:
list(df_review.text)

['total bill for this horrible service? over $8gs. these crooks actually had the nerve to charge us $69 for 3 pills. i checked online the pills can be had for 19 cents each! avoid hospital ers at all costs.',
 "i *adore* travis at the hard rock's new kelly cardenas salon!  i'm always a fan of a great blowout and no stranger to the chains that offer this service; however, travis has taken the flawless blowout to a whole new level!  \n\ntravis's greets you with his perfectly green swoosh in his otherwise perfectly styled black hair and a vegas-worthy rockstar outfit.  next comes the most relaxing and incredible shampoo -- where you get a full head message that could cure even the very worst migraine in minutes --- and the scented shampoo room.  travis has freakishly strong fingers (in a good way) and use the perfect amount of pressure.  that was superb!  then starts the glorious blowout... where not one, not two, but three people were involved in doing the best round-brush action my hair

In [8]:
df_review.text = df_review.text.astype(str)

In [9]:
df_review.dtypes

Unnamed: 0        int64
Unnamed: 0.1     object
review_id        object
user_id          object
business_id      object
rating          float64
useful          float64
funny           float64
cool            float64
text             object
date             object
year            float64
dtype: object

In [10]:
# find data type of each value of dataframe column
def find_type(data):
    data_type = []
    for review in data:
        type1 = type(review)
        data_type.append(type1)
    return data_type

In [11]:
type_list = find_type(df_review.text)
type_list[:10]
set(type_list)

{str}

In [12]:
# check if float type is in type_list
float in type_list

False

In [13]:
# # replace punctuations and symbols with blank
# for review in df_review["text"]:
#     for symbol in ",.?!''\n":
#         review.replace(symbol, '').lower()

In [14]:
import re
def strip_punct(string):
    no_punct_string = re.sub(r'[^\w\s]','', string)
    return no_punct_string

In [15]:
strip_punct(df_review.text[1])

'i adore travis at the hard rocks new kelly cardenas salon  im always a fan of a great blowout and no stranger to the chains that offer this service however travis has taken the flawless blowout to a whole new level  \n\ntraviss greets you with his perfectly green swoosh in his otherwise perfectly styled black hair and a vegasworthy rockstar outfit  next comes the most relaxing and incredible shampoo  where you get a full head message that could cure even the very worst migraine in minutes  and the scented shampoo room  travis has freakishly strong fingers in a good way and use the perfect amount of pressure  that was superb  then starts the glorious blowout where not one not two but three people were involved in doing the best roundbrush action my hair has ever seen  the team of stylists clearly gets along extremely well as its evident from the way they talk to and help one another that its really genuine and not some corporate requirement  it was so much fun to be there \n\nnext trav

In [16]:
# strip punctuations for dataframe column using lambda apply and a function
df_review['text'] = df_review.apply(lambda row: strip_punct(row['text']), axis=1)

In [17]:
# check if text is stripped of punctuations
df_review.text[0]

'total bill for this horrible service over 8gs these crooks actually had the nerve to charge us 69 for 3 pills i checked online the pills can be had for 19 cents each avoid hospital ers at all costs'

In [ ]:
# Import the wordcloud library
from wordcloud import WordCloud
# Join the different processed reviews together.
long_string = ','.join(list(df_reviewiew['text'].values))
# Create a WordCloud object
wordcloud = WordCloud(background_color="white", max_words=5000, contour_width=3, contour_color='steelblue')
# Generate a word cloud
wordcloud.generate(long_string)
# Visualize the word cloud
wordcloud.to_image()

## Word Cloud

In [18]:
# count_vectorizer = CountVectorizer(ngram_range=(1, 2),  
#                                    stop_words='english', 
#                                    token_pattern="\\b[a-z][a-z]+\\b",
#                                    lowercase=True,
#                                    max_df = 0.6, max_features=4000)
# tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2),  
#                                    stop_words='english', 
#                                    token_pattern="\\b[a-z][a-z]+\\b",
#                                    lowercase=True,
#                                    max_df = 0.6, max_features=4000)

# cv_data = count_vectorizer.fit_transform(df_review.text)
# tfidf_data = tfidf_vectorizer.fit_transform(df_review.text)